Calculate Mbar for distance measurements.
17 Dec 2022, J. Jensen
Lightly tested using only a single galaxy and wavelength

In [9]:
import sys, os
import datetime
from settings import *

sys.path.insert(0, PYSBF_PATH)
from pysbf import *

In [10]:
IN_FOLDER, OUT_FOLDER

('/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks/data/wfc3/',
 '/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks/output/')

In [11]:
PS_PATH, IN_FOLDER, CONFIG_FOLDER

('/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks/data/PS/',
 '/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks/data/wfc3/',
 '/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/pysbf/config/')

In [12]:
! ls {DEFAULT_MBAR_CALIB}

/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/pysbf/py/params/mbar_calibrations.yml


In [13]:
! ls {DEFAULT_MBAR_PARAM}

/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/pysbf/py/params/mbar_params.yml


In [14]:
pysbf_path = PYSBF_PATH
galaxy_root = IN_FOLDER
PS_root = PS_PATH
configFolder = CONFIG_FOLDER

In [15]:
with open("u12517_color_ps_gz.json", 'r') as f:
        colors = json.load(f)

colors

{'AVC0': 1.356786847114563,
 'AVC1': 1.362874984741211,
 'AVC2': 1.3603893518447876,
 'AVC5': 1.3620602190494537,
 'Camera': 'PS',
 'Dec': 43.95766666667,
 'MEDC0': 1.3593435287475586,
 'MEDC1': 1.3640217781066895,
 'MEDC2': 1.3630905151367188,
 'MEDC5': 1.3597984611988068,
 'RA': 349.9766666667,
 'Radii': [{'Average_radius': 23.4},
  {'Average_radius': 23.2},
  {'Average_radius': 50.2},
  {'Average_radius': 92.5},
  {'Average_radius': 183.2},
  {'Average_radius': 176.5},
  {'Average_radius': 179.2},
  {'Average_radius': 182.9}],
 'bands': ['g', 'z'],
 'extinctions': [0.592, 0.247],
 'name': 'u12517'}

In [16]:
gz_Mbar = get_Mbar(
         Mbar_filter = "F110W",     # F110W or F160W
         color_camera = "PS",       # PS, SDSS, DECam, ACS, 2MASS, WFC3IR
         calib_mode = "linear",     # linear, quadratic, group, individual; linear default
         calib_color = "g-z",       # g-z or j-h
         color_json = "u12517_color_ps_gz.json",
         verbose=True)

gz_Mbar

c0: (g-z) = 1.475 +/- 0.038   Mbar = -2.701 +/- 0.119
c1: (g-z) = 1.481 +/- 0.038   Mbar = -2.69 +/- 0.119
c2: (g-z) = 1.479 +/- 0.0389   Mbar = -2.692 +/- 0.12
c5: (g-z) = 1.476 +/- 0.0399   Mbar = -2.7 +/- 0.122


{'c0': {'g-z': (1.4754031333923343, 0.03802959373961284),
  'Mbar': (-2.7011292318725575, 0.11892696918697626)},
 'c1': {'g-z': (1.480511781692505, 0.03802959373961284),
  'Mbar': (-2.6900945515441888, 0.11892696918697626)},
 'c2': {'g-z': (1.479494842529297, 0.03891336531321853),
  'Mbar': (-2.6922911401367178, 0.12025341907821165)},
 'c5': {'g-z': (1.4758999196290972, 0.039877938763180826),
  'Mbar': (-2.70005617360115, 0.1217188169512011)}}

In [17]:
# Get SBF data from bestfluc output in json format--or better yet, just add all of this to bestfluc!
# Bestfluc already has all this information, but it lacks the calibration definitions and colors.

# These values for U12517 are for testing.
ext_110 = 0.164

In [18]:
name = "u12517"

In [19]:
OUT_FOLDER

'/media/Data/Home/PanStarrs/Jan/HI/augment/SBF/codes/notebooks/output/'

In [20]:
logFile = OUT_FOLDER+"Outputs_"+name+'/'+name+"_model_log.csv"

df_log = open_log_df(logFile)
sky_med = float(df_log.loc["initial_sky_med"].value)

df = open_log_df(logFile)
obj_uuid = df.loc['uuid'].value

# sky = 3250      # background, eliprof notebook
SKY = df.loc["sky"].value

In [21]:
df.head(7)

,value,description
uuid,eb317d7f178c,Unique Identifier Code
User,Ehsan,User Name
Time,2023-06-06 17:27:11.099097,Modification Time
Name,u12517,Object Name
X_pixels,1022,X-dimension of image [pixel]
Y_pixels,1025,Y-dimension of image [pixel]
R_max,454,maximum horizontal/vertical distance from cent...


In [14]:
# area(k) = pi*(r1**2-r0**2) * abs(a1-a0) / 360


# sliver(k) = bar1(k) / (area(k)/(4*kmax(k)**2))

In [27]:
# rc0 = 6.4       # get_sbf results <r> =  99.5 
# rc1 = 12.7      # the average radius of each anulus
# rc2 = 25.2
# rc5 = 50

# fracc0 = 0.9    # comes from get_sbf - number of unmasked pixels in an anulus comupted by bestfluc (sliver)
# fracc1 = 0.89   # for now I assume it is == 1 
# fracc2 = 0.85
# fracc5 = 0.78

rc_dict = {}

for c in ["c0", "c1", "c2", "c5"]:
    
    try:
        with open(f"power_{c}.txt", 'r') as f:
            lines = f.readlines()
        rc_dict[f"r{c}"] = float(lines[2].split()[9])
    except:
        pass
    
rc_dict

{'rc0': 49.7, 'rc1': 99.5, 'rc2': 198.3}

In [26]:
l[9][:-1]

'49.'

In [28]:
l = lines[1].split()
r0 = float(l[9][:-1])
r1 = float(l[10][:-1])

a0 = float(l[14][:-1].split(",")[0])
a1 = float(l[14][:-1].split(",")[1])

l = lines[2].split()
rc = float(l[9])
bar1 = float(l[3])

kmax = float(lines[-1].split()[0])

area = np.pi*(r1**2-r0**2) * abs(a1-a0) / 360.
frac = sliver = bar1 / (area/(4*kmax**2))

r0, r1, a0, a1, rc, Bar1, kmax, area, sliver

NameError: name 'Bar1' is not defined

In [24]:
with open(f"power_c0.txt", 'r') as f:
    lines = f.readlines()
    
lines

['PSF:  (order,K0,K1) = (4,  0, 15)   Scale = 1.846E+08   Sum/Scale = 1.000\n',
 'MASK: (X0,Y0) = ( 567, 562)   (R0,R1) = (  32,  64)   (A0,A1) = (  0,360)\n',
 'AVG:  <1> = 0.030    <g> = 56418.5    <r> =  49.7    E(k=0) =  1551.3\n',
 'RESIDUAL VARIANCE: m_res =   0.00\n',
 'P0=  2.206E+01 +/-  3.52E-01   P1=  1.553E+00   Kfit=(25, 60)   Cor cf= 0.958\n',
 '\n',
 '   k   Expect(k)   Data(k) Efit/E0 Data/E0   P0*E      P1      P0*E+P1   P0       P1       RMS    Corcof\n',
 '   0.0 1.500E+03 5.168E+05  1.0000 3.33E+02 2.21E+01 1.55E+00 2.36E+01 2.87E+01 7.61E-01 5.90E+03 22.059\n',
 '   1.0 1.498E+03 4.156E+05  0.9901 2.68E+02 2.18E+01 1.55E+00 2.34E+01 2.86E+01 7.73E-01 5.55E+03*******\n',
 '   2.0 1.495E+03 2.462E+05  0.9803 1.59E+02 2.16E+01 1.55E+00 2.32E+01 2.82E+01 8.17E-01 4.44E+03  0.809\n',
 '   3.0 1.490E+03 1.080E+05  0.9706 6.96E+01 2.14E+01 1.55E+00 2.30E+01 2.80E+01 8.51E-01 3.98E+03  0.822\n',
 '   4.0 1.482E+03 3.807E+04  0.9609 2.45E+01 2.12E+01 1.55E+00 2.27E+01 2.79E

In [29]:
with open(f"get_sbf_monsta.log", 'r') as f:  # get one for each anulus
    lines = f.readlines()
    
lines

['Unity = 0.121     Average radius =    99.5\n',
 'Residual radius =    99.5     m_res =  0.00\n',
 'Unity = 0.121     Galaxy average = 19695.7\n',
 ' PSF scaled by 1.846E+08\n',
 ' Mongo commands: power, p0, fluc, etc\n',
 ' 1:k 2:E 3:D 4:Efit 5:R 6:P0 7:P1 8:dP0 9:dP1 10:rms 11:Chi/n\n',
 ' Averaging fits from wavenumbers          25          60\n',
 ' P0 =  2.022E+01 +/-  0.00        P1 =  1.24 +/- 0.00     ( 2.803E+03)\n',
 ' rms in P0 =      0.16        Fit range =  25  60     Correlation r = 0.9879\n',
 'Autoscaling: threshold =  -661.176         saturation =   674.502    \n',
 ' >> \n']

In [30]:
# galskyc0 = 16.4  # ratio of the gal brightness to background
# galskyc1 = 5.1   
# galskyc2 = 1.43
# galskyc5 = 0.335

with open(f"get_sbf_monsta.log", 'r') as f:  # get one for each anulus
    lines = f.readlines()

galaxy_average = float(lines[2].split()[6])   # <g> = 56418.5

outFolder = OUT_FOLDER + "Outputs_"+name+'/'
logFile = outFolder+name+"_model_log.csv"
df = open_log_df(logFile)

sky_background = float(df.loc["sky"].value)

galsky = galaxy_average / sky_background

galsky

6.071424167694205

In [37]:
_ = Logtext(os.path.join("./output/Outputs_u12517/u12517_eb317d7f178c/", "hybridj.lkn6"), "hybrid")

In [22]:
#  Residual variance PER PIXEL in magnitudes, V = 1*f^2
#        bright  faint   Residual variance
#    r      m      m    m_gc   m_gxy  m_all  slope   bias
#    9.0  20.00  24.14  26.87  29.06  26.86   0.00   0.00

# bright mag cutfff -> we specified
# faint_m : == m_gc + m_gxy

# n358 bestflu
#       do 8 i = 1,npred
#          read(1,*) ran(i), emb, emf(i), emgc, emgxy, emres(i)

#          if(npred.gt.1) then
#             emr(k) = emres(j-1) + (emres(j)-emres(j-1)) *
#      $        (barr(k)-ran(j-1))/(ran(j)-ran(j-1))
#             emcut(k) = emf(j-1) + (emf(j)-emf(j-1)) *
#      $        (barr(k)-ran(j-1))/(ran(j)-ran(j-1))
#             emdiff(k) = cmax - emcut(k)
#          end if


In [23]:
# dophot-SE 
GCpeak = 25.0         # mag of the GCLF peak --> likenew notebook
# Cmax  =	      25.805 		 GC peak magnitude


GCpeakdist = 70       # distance that used to compute the GCLF peak


# in the center, it might not be good, 
GCpeakdiffc0 = 0.52   # completness magnitude - GCLF peak : bad if it's a big number (> ~ 0.5 mg)
GCpeakdiffc1 = 0.12
GCpeakdiffc2 = -0.16
GCpeakdiffc5 = -0.34

In [24]:
PrP0c0 = 0.07    # ratio or Pr/P0
PrP0c1 = 0.068
PrP0c2 = 0.097
PrP0c5 = 0.213

snrc0 = 9.65    # (P0-Pr)/P1
snrc1 = 10.3
snrc2 = 8.6
snrc5 = 3.6

In [25]:
mbarc0 = 31.64   # --> P0-Pr   # fluctation magnitudes  <- get_sbf
mbarc1 = 31.59
mbarc2 = 31.51
mbarc5 = 31.46

P0sigc0 = 0.02     # fit uncertainty on P0
P0sigc1 = 0.02
P0sigc2 = 0.03
P0sigc5 = 0.07

skysigc0 = 0.02    # Monte carlo  -> sigma due to sky uncertainty
skysigc1 = 0.03
skysigc2 = 0.05
skysigc5 = 0.10

# psfsigc0 = 0.015
# psfsigc1 = 0.015
# psfsigc2 = 0.015
# psfsigc5 = 0.015

# Prsigc0 = Prsig * PrP0c0 
# Prsigc1 = Prsig * PrP0c1
# Prsigc2 = Prsig * PrP0c2
# Prsigc5 = Prsig * PrP0c5

In [26]:
# # combine observational uncertainties in quadrature
# mbarsigc0 = math.sqrt(P0sigc0**2 + skysigc0**2 + psfsigc0**2 + Prsigc0**2)
# mbarsigc1 = math.sqrt(P0sigc1**2 + skysigc1**2 + psfsigc1**2 + Prsigc1**2)
# mbarsigc2 = math.sqrt(P0sigc2**2 + skysigc2**2 + psfsigc2**2 + Prsigc2**2)
# mbarsigc5 = math.sqrt(P0sigc5**2 + skysigc5**2 + psfsigc5**2 + Prsigc5**2)
    
# print("c0: mbar =", "{:.5}".format(mbarc0)," +/-","{:.2}".format(mbarsigc0))
# print("c1: mbar =", "{:.5}".format(mbarc1)," +/-","{:.2}".format(mbarsigc1))
# print("c2: mbar =", "{:.5}".format(mbarc2)," +/-","{:.2}".format(mbarsigc2))
# print("c5: mbar =", "{:.5}".format(mbarc5)," +/-","{:.2}".format(mbarsigc5))

In [27]:
# conditions to reject result

In [28]:
# # Identify good regions to use
# print("Min of (P0-Pr)/P1 = ",minsnr)
# print("Max GCLF peak - limiting mag = ",maxGCpeakdiff)
# print("Min fraction unmasked pixels = ", minfrac)
# print("Min galaxy/sky =", mingalsky)
# print("Max Pr/P0 = ", maxPrP0)
# print("Max mbar uncertainty = ", maxmbarsig)
# print("Min (g-z) =",mingz)
# print("Min (J-H) =",minjh)
# print("")
# usec0=False; usec1=False; usec2=False; usec5=False

# if snrc0>minsnr and GCpeakdiffc0<maxGCpeakdiff and fracc0>minfrac and galskyc0>mingalsky and PrP0c0<maxPrP0 and mbarsigc0<maxmbarsig and acs_gzc0>mingz:
#     usec0=True
#     print("Including c0")
# if snrc1>minsnr and GCpeakdiffc1<maxGCpeakdiff and fracc1>minfrac and galskyc1>mingalsky and PrP0c1<maxPrP0 and mbarsigc1<maxmbarsig and acs_gzc1>mingz:
#     usec1=True
#     print("Including c1")
# if snrc2>minsnr and GCpeakdiffc2<maxGCpeakdiff and fracc2>minfrac and galskyc2>mingalsky and PrP0c2<maxPrP0 and mbarsigc2<maxmbarsig and acs_gzc2>mingz:
#     usec2=True
#     print("Including c2")
# if snrc5>minsnr and GCpeakdiffc5<maxGCpeakdiff and fracc5>minfrac and galskyc5>mingalsky and PrP0c5<maxPrP0 and mbarsigc5<maxmbarsig and acs_gzc5>mingz:
#     usec5=True
#     print("Including c5")
   

In [79]:
# # Calculate distances and uncertainties

# distmodc0 = (mbarc0 - Mbarc0)
# distmodc1 = (mbarc1 - Mbarc1)
# distmodc2 = (mbarc2 - Mbarc2)
# distmodc5 = (mbarc5 - Mbarc5)

# distmodsigc0 = math.sqrt(mbarsigc0**2 + Mbarsigc0**2)
# distmodsigc1 = math.sqrt(mbarsigc1**2 + Mbarsigc1**2)
# distmodsigc2 = math.sqrt(mbarsigc2**2 + Mbarsigc2**2)
# distmodsigc5 = math.sqrt(mbarsigc5**2 + Mbarsigc5**2)

# # these are calculated here for reference but are not used later.
# distc0 = 10**(distmodc0/5 - 5)
# distc1 = 10**(distmodc1/5 - 5)
# distc2 = 10**(distmodc2/5 - 5)
# distc5 = 10**(distmodc5/5 - 5)

# distsigc0 = (10**((distmodc0+distmodsigc0)/5 - 5) - 10**((distmodc0-distmodsigc0)/5 - 5)) / 2
# distsigc1 = (10**((distmodc1+distmodsigc1)/5 - 5) - 10**((distmodc1-distmodsigc1)/5 - 5)) / 2
# distsigc2 = (10**((distmodc2+distmodsigc2)/5 - 5) - 10**((distmodc2-distmodsigc2)/5 - 5)) / 2
# distsigc5 = (10**((distmodc5+distmodsigc5)/5 - 5) - 10**((distmodc5-distmodsigc5)/5 - 5)) / 2

# n=0
# if usec0:
#     n=n+1
#     print("c0: (m-M) =", "{:.5}".format(distmodc0)," +/-","{:.3}".format(distmodsigc0),"  d =","{:.3}".format(distc0)," +/-","{:.2}".format(distsigc0)," Mpc")
# if usec1:
#     n=n+1
#     print("c1: (m-M) =", "{:.5}".format(distmodc1)," +/-","{:.3}".format(distmodsigc1),"  d =","{:.3}".format(distc1)," +/-","{:.2}".format(distsigc1)," Mpc")
# if usec2:
#     n=n+1
#     print("c2: (m-M) =", "{:.5}".format(distmodc2)," +/-","{:.3}".format(distmodsigc2),"  d =","{:.3}".format(distc2)," +/-","{:.2}".format(distsigc2)," Mpc")
# if usec5:
#     n=n+1
#     print("c5: (m-M) =", "{:.5}".format(distmodc5)," +/-","{:.3}".format(distmodsigc5),"  d =","{:.3}".format(distc5)," +/-","{:.2}".format(distsigc5)," Mpc")

# # write distances and uncertainties to the output file

In [80]:
# # Weighted average distances for good annuli only

# # Note that this section trashes the computed values for the excluded regions, so you need to rerun the code from the beginning.
# if not usec0:
#     distmodc0=0; distmodsigc0=1E9; mbarsigc0=1E9; Mbarsigc0=0
# if not usec1:
#     distmodc1=0; distmodsigc1=1E9; mbarsigc1=1E9; Mbarsigc1=0
# if not usec2:
#     distmodc2=0; distmodsigc2=1E9; mbarsigc2=1E9; Mbarsigc2=0
# if not usec5:
#     distmodc5=0; distmodsigc5=1E9; mbarsigc5=1E9; Mbarsigc5=0

# # annular regions are treated as independent measurements for weighted average and uncertainties
# distmodwtav = math.sqrt(distmodc0**2/distmodsigc0**2 + distmodc1**2/distmodsigc1**2 + distmodc2**2/distmodsigc2**2 + distmodc5**2/distmodsigc5**2) / math.sqrt(1/distmodsigc0**2 + 1/distmodsigc1**2 + 1/distmodsigc2**2 + 1/distmodsigc5**2)
# mbarsigwtav = 1 / math.sqrt(1/mbarsigc0**2 + 1/mbarsigc1**2 + 1/mbarsigc2**2 + 1/mbarsigc5**2)

# # Extintion uncertainty is not independent for annuli, it's systematic, so add it on
# mbarsigwtav = (ext_sig * ext) + mbarsigwtav  # add systematic uncertainty from extinction to obs error

# # Calibration Mbar uncertainties are not really independent for each region. Just average them.
# Mbarsigwtav = (Mbarsigc0 + Mbarsigc1 + Mbarsigc2 + Mbarsigc5)/n


# distmodsigwtav = math.sqrt(mbarsigwtav**2 + Mbarsigwtav**2)  # add obs and cal errors in quadrature

# distwtav = 10**(distmodwtav/5 - 5)
# # Distance uncertainties are not symmetrical in distance. I just average the + and - uncertaintes...
# distsigwtavplus = 10**((distmodwtav+distmodsigwtav)/5 -5) - distwtav
# distsigwtavminus = distwtav - 10**((distmodwtav-distmodsigwtav)/5 -5)
# distsigwtav = (distsigwtavplus + distsigwtavminus) / 2

# print("(m-M) =","{:.5}".format(distmodwtav),"+/-","{:.3}".format(distmodsigwtav))
# print("d =","{:.3}".format(distwtav),"+/-","{:.2}".format(distsigwtav))
# print("d =","{:.3}".format(distwtav),"+","{:.2}".format(distsigwtavplus),"-","{:.2}".format(distsigwtavminus))

# # Write results to the output file.


In [81]:
# more ./wfc3-16262/u11990/u11990doj.MBAR 
# the output ogf bestfluc